# Práctica 1

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker
import tensorflow as tf
import random
import time
import collections

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Quitar warnings feos
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocesamiento de datos

### Creación del dataset


In [2]:
#leemos los datos aqui
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')
iris_OG = iris_RAW.copy()
salidas = iris_RAW.pop("class")

## Creación entradas y salidas

In [3]:
entradas = iris_RAW.to_numpy()
mm = MinMaxScaler()
entradas = mm.fit_transform(entradas)

"""
# Categorizamos salidas
label_encoder = LabelEncoder()
salidas = label_encoder.fit_transform(salidas)
"""
salidas = pd.get_dummies(salidas)

# Funciones algoritmo genético

### Variables a definir

In [4]:
"""
ngenes: nneuronas de cada individuo. Sacado a traves del L4P1-EstudioRed.ipynb
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
CONVGEN: Número de generaciones que si no hay cambio para.
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
"""

ngenes = 7 # NO TOCAR
npob = 20 # NO TOCAR
ngeneraciones = 200 # TOCAR
convgen = 10 # TOCAR
pm = 0.1 # TOCAR
nres = 10 # TOCAR
nsample = 30 # TOCAR

### Crear inidividuo

Como bien se dice en el enunciado, un individuo es un MLP sin entrenar, con tantos genes como hemos calculado. En nuestro caso son 7 genes, y el tamaño de entrada son 4. De esta manera creamos un MLP básico.

In [5]:
def crearMLP(ngenesMLP, tam_entrada,tipo,pesos):

    MLP = ker.Sequential() # Creamos el modelo vacío

    # Añadimos la primera capa (tecnicamente dos, la de entrada (con la variable input_dim) y la oculta)
    MLP.add(ker.layers.Dense(ngenesMLP, input_dim = tam_entrada, activation='relu'))

    # Añadimos la capa de salida
    MLP.add(ker.layers.Dense(3,activation='sigmoid')) 

    # Por último tenemos que configurar el modelo antes de entrenarlo con lo dicho en el enunciado
    # Optimizador: Adam
    # Función error: Mean Squared Error
    # Metricas = Binary accuracy porque nos da el número de precisión de la red
    MLP.compile(optimizer='adam',loss='mean_squared_error',metrics=["accuracy"])

    if(tipo == 1): # Le metemos pesos ya predeterminados
        MLP.set_weights(pesos)

    return MLP

### Crear población

Al igual que antes hemos creado un individuo en este caso crearemos una población. Una población es un conjunto de individuos, en nuestro caso entonces realizaremos un array de población

In [6]:
def crearPoblacion(tamPoblacion,ngenesMLP, tam_entrada):
    poblacion = []
    for i in range (tamPoblacion):
        time.sleep(1)
        individuo = crearMLP(ngenesMLP,tam_entrada,0,0)
        poblacion.append(individuo)
    return poblacion

### Evaluar población

El calculo del fitness de la población se realiza mediante el evaluate del MLP. De esta manera, simplemente tendremos que iterar toda la población y devolver un array con el fitness de cada individuo en la posición de la población.

In [7]:
def evaluarPoblacion(poblacion,entradas_test,esperadas_test):
    evalPobl = []
    for individuo in poblacion:
        metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
        evalPobl.append(metricas_totales*150)
    return evalPobl

### Evaluar individuo

Adicionalmente por si es requerido, crearemos una evaluacion para un unico individuo

In [8]:
def evaluarIndividuo(individuo,entradas_test,esperadas_test):
    metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
    evalInd = metricas_totales*150
    return evalInd

### Selección de replicar individuo

Posteriormente necesitaremos un algoritmo de montecarlo, que nos permita elegir la probablididad de selección de el individuo. El método de montecarlo es generar un número aleatorio entre [0,1], si este número es menor que la probabilidad de selección, el individuo se replica, si no, no. En nuestro caso, recibimos una población y la recorremos, viendo cuales se cogen o no. De esta manera, devolvemos un de booleano donde indicamos si ese individuo se replica o no.

In [9]:
def seleccion(individuo,poblacion,entradas_test,esperadas_test):
    fis_totales = evaluarPoblacion(poblacion,entradas_test,esperadas_test)
    fi = evaluarIndividuo(individuo,entradas_test,esperadas_test)
    nAleatorio = random.uniform(0, 1)
    ps = fi / np.amax(fis_totales)
    return nAleatorio < ps

### Mutar

Secuencialmente para todos los genes del padre decide si cada gen se copia fielmente en el hijo o se muta usando un Método de Montecarlo simple sobre el valor de Pm. Si muta, cámbialo por otro valor aleatorio en el rango (-1, 1). Prueba también otras opciones de mutación. 

In [11]:
def mutar(individuo):
    pesos_antiguos = individuo.get_weights()
    pesos_entrada_oculta = pesos_antiguos[0]
    pesos_oculta_salida = pesos_antiguos[2]

    for i in range(len(pesos_entrada_oculta)):
        for j in range(len(pesos_entrada_oculta[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_entrada_oculta[i][j] = random.uniform(-1, 1)

    for i in range(len(pesos_oculta_salida)):
        for j in range(len(pesos_oculta_salida[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_oculta_salida[i][j] = random.uniform(-1, 1)
    
    aux1 = np.zeros(ngenes, dtype='float32')
    aux2 = np.array([0.,0.,0.], dtype='float32')
    pesos_nuevos = [pesos_entrada_oculta,aux1,pesos_oculta_salida,aux2] # Los pesos con mutación

    nuevoMLP = crearMLP(ngenes,4,1,pesos_nuevos)

    return nuevoMLP

### Resumen

In [12]:
def resumen(poblacion, entradas, salidas, generacion):
    pobFit = evaluarPoblacion(poblacion, entradas, salidas)
    valMejInd = np.amax(pobFit)
    nInd = collections.Counter(pobFit)[valMejInd]
    return valMejInd,str((nInd/npob)*100)


### Simulación

In [14]:
def simular():

    fitantiguo = 0
    fitnuevo = 0
    
    poblacion = crearPoblacion(npob,ngenes, 4) # Población inicial

    ultimaGen = 0 # Numero de generacion necesaria para obtener el resultado optimo 
    ultimoFitness = 0 # Mejor fitness posible con esta configuracion de
    ultimoPorcMejoresInd = 0 # Ultimo porcentaje de mejores individuos

    fitnes_antiguo = 0
    num_conv = 0 # numero de veces que el fitnes se ha repetido de
    lim_conv = 5 # limite de veces que el fitnes se puede repetir para la condicion de terminacion                              TOCAR

    for i in range(ngeneraciones):
        mutaciones_cont = 0
        #print("Generacion ", str(i+1), end="\r")
        for individuo in poblacion: # Recorremos toda la población
            seleccionado = seleccion(individuo,poblacion,entradas,salidas) # Vemos si le toca mutarse o no
            if(seleccionado): # Lo mutamos y matamos alguien aleatorio
                nuevoIndividuo = mutar(individuo) # Lo mutamos
                muerto = random.randint(0, npob-1) # Nos cargamos un random
                poblacion[muerto] = nuevoIndividuo
                mutaciones_cont += 1

        fitnes, porcentaje = resumen(poblacion, entradas, salidas, i)
        print(f"Generacion {str(i+1)} -> fitnes {fitnes}")
        ultimoFitness = fitnes
        ultimoPorcMejoresInd = porcentaje
        if(fitnes_antiguo == fitnes):
            num_conv += 1
        else:
            num_conv = 0
            fitnes_antiguo = fitnes 

        # Condicion de parada
        
        ultimaGen = i
        if(num_conv == lim_conv):
            return [ultimaGen,ultimoFitness, ultimoPorcMejoresInd]

    return [ultimaGen, ultimoFitness, ultimoPorcMejoresInd]      

## Comienza el automatizador
</br>
Primero inicializamos todas las variables necesarias

In [15]:
ngeness = [i for i in range(0,11,2) if i != 0] # AUTOMATIZAR
npobs = [i for i in range(0,51,10) if i != 0] # AUTOMATIZAR
ngeneraciones = 200 # TOCAR
convgen = 10 # TOCAR
pms = [0.01,0.05,0.1,0.15,0.2,0.25,0.3] # AUTOMATIZAR
nres = 10 # TOCAR
nsample = 30 # TOCAR

Y ahora realizamos los bucles anidados

In [16]:
salidaOG = [0,0,0] # guarda el numero de generaciones, el fitnes y el porcentaje individuos con ese fitnes del mejor algoritmo
solucion = [] # guarda que valor de npob y pm corresponde con salidaOG
for i in ngeness:
    for j in npobs:
        print("=================================================================================")
        print(f"Probando con el numero de genes = {i} y el numero de poblaciones = {j}")
        print("=================================================================================")
        # Ahora vamos a realizar el algorimo genético con los valores dados y vemos que nos da para compararlo
        ngnenes = i 
        npob = j
        salidaN = simular() # guardamos los valores en la salida Nueva
        if salidaOG[1] < salidaN[1]: # 1 -> fitnes del mejor individuo  
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]}")
            salidaOG = salidaN
            solucion = [i,j]
        elif salidaOG[1] == salidaN[1]: # 0 -> Numero de generaciones
            if salidaOG[0] > salidaN[0]: # si el fitnes es igual, pero la nueva salida lo ha conseguido con menos generaciones, esta sera la nueva mejor
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el numero de generaciones, Antiguo: {salidaN[0]} Nuevo: {salidaOG[0]}")
                salidaOG = salidaN
                solucion = [i,j]
        elif salidaOG[0] == salidaN[0] and salidaOG[1] == salidaN[1]: # si ambos valores son iguales, se busca aquel con mayor porcentaje de elementos con el mejor fitness
            if salidaOG[2] < salidaN[2]: #2 -> % de individuos con mejor fitness
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el % de individuos con mejor fitnes, Antiguo: {salidaN[2]} Nuevo: {salidaOG[2]}")
                salidaOG = salidaN
                solucion = [i,j]
print(f"La combinacion perfecta encontrada es de numero de poblacion = {solucion[0]} y probabilidad de mutacion = {solucion[1]}")
print(f"Dando como resultado un fitnes de {salidaOG[1]}/150, con un numero de generaciones de {salidaOG[0]} y un porcentaje de individuos con ese fitnes de {salidaOG[2]}")

Probando con el numero de genes = 2 y el numero de poblaciones = 10
Generacion 1 -> fitnes 91.05
Generacion 2 -> fitnes 97.95
Generacion 3 -> fitnes 97.95
Generacion 4 -> fitnes 97.95
Generacion 5 -> fitnes 96.0
Generacion 6 -> fitnes 96.0
Generacion 7 -> fitnes 96.0
Generacion 8 -> fitnes 51.00000000000001
Generacion 9 -> fitnes 49.95
Generacion 10 -> fitnes 49.95
Generacion 11 -> fitnes 79.05
Generacion 12 -> fitnes 49.95
Generacion 13 -> fitnes 49.95
Generacion 14 -> fitnes 63.0
Generacion 15 -> fitnes 97.95
Generacion 16 -> fitnes 67.05
Generacion 17 -> fitnes 64.05
Generacion 18 -> fitnes 49.95
Generacion 19 -> fitnes 52.949999999999996
Generacion 20 -> fitnes 51.00000000000001
Generacion 21 -> fitnes 51.00000000000001
Generacion 22 -> fitnes 63.0
Generacion 23 -> fitnes 63.0
Generacion 24 -> fitnes 94.95
Generacion 25 -> fitnes 99.0
Generacion 26 -> fitnes 99.0
Generacion 27 -> fitnes 100.05000000000001
Generacion 28 -> fitnes 100.95
Generacion 29 -> fitnes 100.05000000000001
Gen

In [ ]:
print("Ahora buscamos la relacion entre esos valores y la probilidad de mutacion")
solucion = np.append(pm)
for i in pms:
    pm = i
    ngenes = solucion[0]
    npob = solucion[1]
    print("=========================================================")
    print(f"Probando con la probabilidad de mutacion = {i}")
    print("=========================================================")
    salidaN = simular() # guardamos los valores en la salida Nueva
    if salidaOG[1] < salidaN[1]: # 1 -> fitnes del mejor individuo  
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]}")
            salidaOG = salidaN
            solucion = [solucion[0],solucion[1],i]
    elif salidaOG[1] == salidaN[1]: # 0 -> Numero de generaciones
        if salidaOG[0] > salidaN[0]: # si el fitnes es igual, pero la nueva salida lo ha conseguido con menos generaciones, esta sera la nueva mejor
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el numero de generaciones, Antiguo: {salidaN[0]} Nuevo: {salidaOG[0]}")
                salidaOG = salidaN
                solucion = [solucion[0],solucion[1],i]
    elif salidaOG[0] == salidaN[0] and salidaOG[1] == salidaN[1]: # si ambos valores son iguales, se busca aquel con mayor porcentaje de elementos con el mejor fitness
        if salidaOG[2] < salidaN[2]: #2 -> % de individuos con mejor fitness
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el % de individuos con mejor fitnes, Antiguo: {salidaN[2]} Nuevo: {salidaOG[2]}")
                salidaOG = salidaN
                solucion = [solucion[0],solucion[1],i]
print(f"Por lo tanto, la red ejecutada que mejor ha funcionado tiene:\n Nº de genes = {solucion[0]}\n Nº de poblaciones = {solucion[1]}\n Probabilidad de mutacion = {solucion[2]}")
print(f"Dando como resultado un fitnes de {salidaOG[1]}/150, con un numero de generaciones de {salidaOG[0]} y un porcentaje de individuos con ese fitnes de {salidaOG[2]}")